In [ ]:
%pip install pandas

### 1. Load data

In [29]:
import pandas as pd
train_df = pd.read_csv('../data/vn30/ACB_train.csv')
train_df.head(10)

,time,open,high,low,close,volume
0,2020-01-02,7.88,7.94,7.84,7.94,1163109
1,2020-01-03,7.94,8.01,7.91,7.94,1055528
2,2020-01-06,7.94,7.94,7.77,7.77,1286035
3,2020-01-07,7.77,7.84,7.77,7.77,1050934
4,2020-01-08,7.77,7.77,7.60,7.63,2304937
5,2020-01-09,7.67,7.81,7.67,7.74,1422248
6,2020-01-10,7.77,7.98,7.77,7.84,2371501
7,2020-01-13,7.91,7.91,7.84,7.91,1523181
8,2020-01-14,7.91,8.12,7.91,8.12,2851982
9,2020-01-15,8.12,8.12,8.01,8.05,1158657


### 2. Proces Data

In [31]:
train_df_1 = train_df[['close', 'volume']]
# # Tính ma trận tương quan
# cor_df = train_df.drop(['Date'], axis=1)
# correlation_matrix = cor_df.corr()

# # Hiển thị ma trận tương quan
# print(correlation_matrix)

In [38]:
def lag_transform_df(df: pd.DataFrame, lag: int):
    df_lag = df.copy()
    for col in df_lag.columns:
        for i in range(1, lag + 1):
            df_lag[f'{col}_lag_{i}'] = df_lag[col].shift(i)
        df_lag[f'{col}_mean'] = df_lag[col].rolling(window=lag).mean()
        df_lag[f'{col}_max'] = df_lag[col].rolling(window=lag).max()
        df_lag[f'{col}_min'] = df_lag[col].rolling(window=lag).min()
    return df_lag.dropna()

df_lag = lag_transform_df(train_df_1, 5)
print(df_lag.shape)
df_lag.head()

,close,volume,close_lag_1,close_lag_2,close_lag_3,close_lag_4,close_lag_5,close_mean,close_max,close_min,volume_lag_1,volume_lag_2,volume_lag_3,volume_lag_4,volume_lag_5,volume_mean,volume_max,volume_min
5,7.74,1422248,7.63,7.77,7.77,7.94,7.94,7.770,7.94,7.63,2304937.0,1050934.0,1286035.0,1055528.0,1163109.0,1423936.4,2304937.0,1050934.0
6,7.84,2371501,7.74,7.63,7.77,7.77,7.94,7.750,7.84,7.63,1422248.0,2304937.0,1050934.0,1286035.0,1055528.0,1687131.0,2371501.0,1050934.0
7,7.91,1523181,7.84,7.74,7.63,7.77,7.77,7.778,7.91,7.63,2371501.0,1422248.0,2304937.0,1050934.0,1286035.0,1734560.2,2371501.0,1050934.0
8,8.12,2851982,7.91,7.84,7.74,7.63,7.77,7.848,8.12,7.63,1523181.0,2371501.0,1422248.0,2304937.0,1050934.0,2094769.8,2851982.0,1422248.0
9,8.05,1158657,8.12,7.91,7.84,7.74,7.63,7.932,8.12,7.74,2851982.0,1523181.0,2371501.0,1422248.0,2304937.0,1865513.8,2851982.0,1158657.0


In [40]:
Y = df_lag[['close', 'volume']].values
X = df_lag.drop(['close', 'volume'], axis=1).values
print(X.shape)
print(Y.shape)

(990, 16)
(990, 2)
[7.6300000e+00 7.7700000e+00 7.7700000e+00 7.9400000e+00 7.9400000e+00
 7.7700000e+00 7.9400000e+00 7.6300000e+00 2.3049370e+06 1.0509340e+06
 1.2860350e+06 1.0555280e+06 1.1631090e+06 1.4239364e+06 2.3049370e+06
 1.0509340e+06]
[7.740000e+00 1.422248e+06]


Shuffle and train test split

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)






(792, 16)
(198, 16)
(792, 2)
(198, 2)


In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

model = LinearRegression()
model.fit(X_train, Y_train)

mse = mean_squared_error(Y_test, model.predict(X_test))
print(f"Mean Squared Error: {mse}")

mape = mean_absolute_percentage_error(Y_test, model.predict(X_test))
print(f"Mean Absolute Percentage Error: {mape}")

r2 = r2_score(Y_test, model.predict(X_test))
print(f"R2 Score: {r2}")





Mean Squared Error: 2.3506484482954346e-16
Mean Absolute Percentage Error: 4.618916438637036e-11
R2 Score: 1.0
